In [23]:
from TradeRES_bbresults import BackboneScenarioAnalysis
from TradeRES_bbresults import BackboneResult
from gams import GamsWorkspace
import pandas as pd
import os
from pathlib import Path
import subprocess as sp
from io import StringIO

In [24]:
#load results using David Huckebrink's BackboneResult class
res=BackboneResult("C:\ws\TradeRES\TradeRES-Backbone-demo\.spinetoolbox\items/backbone_loop\output/240205_CfDPaper/__Target.gdx")
res_invest=BackboneResult("C:\ws\TradeRES\TradeRES-Backbone-demo\.spinetoolbox\items/backbone_loop\output/240205_CfDPaper/invest/__Target.gdx")

--- Warning: The GAMS version (43.4.1) differs from the API version (40.3.0).
--- Warning: The GAMS version (43.4.1) differs from the API version (40.3.0).


In [25]:
###Load required target scenario results parameters
#time series of marginals of electricity balance equations by node = wholesale electricity prices
r_balanceMarginal = res.r_balanceMarginal()
r_balanceMarginal=r_balanceMarginal.rename(columns={"Val": "price"})
r_balanceMarginal["price"]=r_balanceMarginal["price"]*-1

#electricity generation time series by unit
r_gen=res.r_gen()
r_gen=r_gen.rename(columns={"Val": "gen_t"})
r_gnuTotalGen = res.r_gnuTotalGen()
r_gnuTotalGen=r_gnuTotalGen.rename(columns={"Val": "gen"})

#cost parameters and installed capacities by unit
p_gnu_io=res.param_as_df("p_gnu_io")
output=p_gnu_io[p_gnu_io['input_output']=='output']
#probably delete: H2=p_gnu_io[p_gnu_io['unit'].apply(lambda x: 1 if any(i in x for i in ["H2 storage"]) else 0)==1]
#p_gnu_io=pd.concat([H2,output])
p_gnu_io["capacity"]=p_gnu_io[p_gnu_io['param_gnu']=='capacity']["value"]
p_gnu_io["fomCosts"]=p_gnu_io[p_gnu_io['param_gnu']=='fomCosts']["value"]
p_gnu_io["vomCosts"]=p_gnu_io[p_gnu_io['param_gnu']=='vomCosts']["value"]
p_gnu_io["invCosts"]=p_gnu_io[p_gnu_io['param_gnu']=='invCosts']["value"]
p_gnu_io["annuityFactor"]=p_gnu_io[p_gnu_io['param_gnu']=='annuityFactor']["value"]
p_gnu_io["unitSize"]=1
p_gnu_io=p_gnu_io.groupby(['node','grid','unit']).sum().reset_index()
wind=p_gnu_io[p_gnu_io['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]

#correct extreme prices (maximum price = vomCosts of industrial load shedding)
unitUnittype=res.set_as_df("unitUnittype")
p_gnu_io=p_gnu_io.merge(unitUnittype,on="unit")
dsr=p_gnu_io[p_gnu_io["unittype"]=="dsr"]
dsr=dsr[["node","vomCosts"]]
dsr=dsr.rename(columns={"vomCosts": "maxprice"})
r_balanceMarginal=r_balanceMarginal.merge(dsr,on=["node"])
r_balanceMarginal["price"][r_balanceMarginal["price"]>r_balanceMarginal["maxprice"]]=r_balanceMarginal["maxprice"]

#merge dataframes and calculate revenues for all new wind onshore power plants
r_gen=r_gen.merge(unitUnittype,on="unit")
r_gen=r_gen[r_gen['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
df_t=r_gen.merge(r_balanceMarginal, on=['t', 'node', 'grid', 'f'])
df_t["rev_t"]=df_t["price"]*df_t["gen_t"]

#create dataframe with total revenues, annual costs, market values (MV), LCOE etc. for each new wind onshore power plant
df_sum=df_t.groupby(['unit','node','grid']).sum().reset_index()
df_sum=df_sum.drop(["maxprice","t","price"],axis=1)
df_sum=df_sum.rename(columns={"rev_t": "rev"})
df_sum=df_sum.rename(columns={"gen_t": "gen"})
df_sum=df_sum.query("grid == 'elec'")
df_sum=df_sum.merge(wind, on=['unit', 'node','grid'])
df_sum["mv"]=df_sum["rev"]/df_sum["gen"]
df_sum["ann_costs"]=(df_sum["annuityFactor"]*df_sum["invCosts"]+df_sum["fomCosts"])*df_sum["capacity"]+df_sum["vomCosts"]*df_sum["gen"]
df_sum["lcoe"]=(df_sum["annuityFactor"]*df_sum["invCosts"]+df_sum["fomCosts"])*df_sum["capacity"]/df_sum["gen"]+df_sum["vomCosts"]

#calculate average revenues per new wind capacity and generation by node
df_sum2=df_sum.groupby(['node','grid']).sum().reset_index()
df_sum2["node_mv"]=df_sum2["rev"]/df_sum2["gen"]
df_sum2["ref_rev"]=df_sum2["rev"]/df_sum2["capacity"]
df_sum2=df_sum2[["node","node_mv","ref_rev"]]
df_sum=pd.merge(df_sum,df_sum2,on="node")


C:\ws\Temp\1\ipykernel_9664\3884728653.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  p_gnu_io=p_gnu_io.groupby(['node','grid','unit']).sum().reset_index()
C:\ws\Temp\1\ipykernel_9664\3884728653.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_balanceMarginal["price"][r_balanceMarginal["price"]>r_balanceMarginal["maxprice"]]=r_balanceMarginal["maxprice"]
C:\ws\Temp\1\ipykernel_9664\3884728653.py:43: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which shou

In [26]:
###Create excel for CfD simulations
#index sheet
ind=[]
ind.append(['Par','p_gnu_io','p_gnu_io!A1',4,1,''])
index=pd.DataFrame(ind,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

ind2=[]
ind2.append(['Par','p_unit','p_unit!A1',1,1,''])
index2=pd.DataFrame(ind2,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

ind3=[]
ind3.append(['Par','p_groupPolicy','p_groupPolicy!A2',2,'',''])
index3=pd.DataFrame(ind3,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

ind4=[]
ind4.append(['Par','p_groupPolicyUnit','p_groupPolicyUnit!A2',3,'',''])
index4=pd.DataFrame(ind4,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

ind5=[]
ind5.append(['Set','ugroup','ugroup!A2',2,'',''])
index5=pd.DataFrame(ind4,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

ind6=[]
ind6.append(['Set','group','group!A2',1,'',''])
index6=pd.DataFrame(ind6,columns=['Type',	'Symbol',	'Rng',	'Rdim', 'Cdim',	'Values'])

index=pd.concat([index,index2,index4,index,index5,index6]).reset_index(drop=True)

In [27]:
###p_gnu_io sheet
df_sum["io"]="output"
df_sum["conversionCoeff"]="x"
df_sum["conversionCoeff"]=1
df_sum['alternative']='x'
df_sum["unitSize"]=1

#1way CfD 
df_md1 =df_sum[['grid','node','unit','io','alternative','conversionCoeff','capacity',
  'unitSize','invCosts', 'fomCosts', 'vomCosts','annuityFactor']]
df_md1["alternative"]="1wayCfD"
df_md1["vomCosts"]=df_sum["node_mv"]-(df_sum["lcoe"]+df_sum["node_mv"]-df_sum["mv"])*1.07+1.3
df_md1["vomCosts"]=np.where(df_md1["vomCosts"]>0,1.3,df_md1["vomCosts"])
df_md1["capacity"]=""

#2way CfD 
df_md2 =df_sum[['grid','node','unit','io','alternative','conversionCoeff','capacity',
  'unitSize','invCosts', 'fomCosts', 'vomCosts','annuityFactor']]
df_md2["alternative"]="2wayCfD"
df_md2["vomCosts"]=df_sum["node_mv"]-(df_sum["lcoe"]+df_sum["node_mv"]-df_sum["mv"])*1.07+1.3
df_md2["capacity"]=""

#Basic CfD
df_md3 = df_sum[['grid','node','unit','io','alternative','conversionCoeff','capacity',
  'unitSize','invCosts', 'fomCosts', 'vomCosts','annuityFactor']]
df_md3["alternative"]="SimpleCfD"
df_md3["invCosts"]=df_sum["invCosts"]-(df_sum["lcoe"]*1.07*df_sum["gen"]-df_sum["rev"])/df_sum["capacity"]/df_sum["annuityFactor"]
df_md3["vomCosts"]=0
df_md3["capacity"]=""

#Financial CfD
df_md4 = df_sum[['grid','node','unit','io','alternative','conversionCoeff','capacity',
  'unitSize','invCosts', 'fomCosts', 'vomCosts','annuityFactor']]
df_md4["alternative"]="FinancialCfD"
df_md4["invCosts"]=df_sum["invCosts"]-((df_sum["lcoe"]*df_sum["gen"]/df_sum["capacity"]-
                                        df_sum["rev"]/df_sum["capacity"]+df_sum["ref_rev"])*1.07-df_sum["ref_rev"])/df_sum["annuityFactor"]
df_md4["capacity"]=""
p_gnu_io_new=pd.concat([df_md1,df_md2,df_md3,df_md4]).reset_index(drop=True)

C:\ws\Temp\1\ipykernel_9664\1510030126.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_md1["alternative"]="1wayCfD"
C:\ws\Temp\1\ipykernel_9664\1510030126.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_md1["vomCosts"]=df_sum["node_mv"]-(df_sum["lcoe"]+df_sum["node_mv"]-df_sum["mv"])*1.07+1.3
C:\ws\Temp\1\ipykernel_9664\1510030126.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [8]:
###p_groupPolicy to fix wind investments of 2 units
p_groupPolicy=p_gnu_io[["unit","capacity"]]
p_groupPolicy=p_groupPolicy[p_groupPolicy['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
p_groupPolicy["unit"]=p_groupPolicy["unit"].map(lambda x: str(x)[:7])
p_groupPolicy=p_groupPolicy.rename(columns={"unit": "group"})
p_groupPolicy=p_groupPolicy.groupby(["group"]).sum().reset_index()
p_groupPolicy=p_groupPolicy.rename(columns={"capacity": " "})
p_groupPolicy["alternative"]="CfD_windfix"
p_groupPolicy["param_policy"]="constrainedCapTotalMax"
p_groupPolicy=p_groupPolicy[["group","alternative","param_policy"," "]]

In [9]:
###group sheet
group=p_gnu_io[["unit"]]
group=group[group['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
group["unit"]=group["unit"].map(lambda x: str(x)[:7])
group=group.rename(columns={"unit": "group"})

In [10]:
###ugroup sheet
ugroup=p_gnu_io[["unit"]]
ugroup=ugroup[ugroup['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
ugroup["group"]=ugroup["unit"].map(lambda x: str(x)[:7])
ugroup=ugroup[["group","unit"]]

In [11]:
###p_GroupPolicyUnit sheet
#maximum wind capacity
p_groupPolicyUnit=p_gnu_io[["unit"]]
p_groupPolicyUnit=p_groupPolicyUnit[p_groupPolicyUnit['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
p_groupPolicyUnit["group"]=p_groupPolicyUnit["unit"].map(lambda x: str(x)[:7])
p_groupPolicyUnit["param_policy"]="constrainedCapMultiplier"
p_groupPolicyUnit["value"]=1
p_groupPolicyUnit["alternative"]="CfD_windfix"
p_groupPolicyUnit=p_groupPolicyUnit[["group","param_policy","unit","alternative","value"]]
#remove vRE constraint
p_groupPolicy2=pd.DataFrame()
p_groupPolicy2.loc[0,"group"]="VRE_limit"
p_groupPolicy2["alternative"]="CfD"
p_groupPolicy2["param_policy"]="energyShareMin"
p_groupPolicy2[" "]=0
p_groupPolicy2=p_groupPolicy2.reset_index(drop=True)
p_groupPolicy=pd.concat([p_groupPolicy,p_groupPolicy2])

In [20]:
###p_unit sheet
#investments in wind onshore profile 2 for the basic CfD case were disabled manually
p_unit=p_gnu_io[["unit","capacity"]]
p_unit1=p_unit[p_unit['unit'].apply(lambda x: 1 if any(i in x for i in ["Wind Onshore New"]) else 0)==1]
p_unit1=p_unit1[p_unit1["capacity"]==0]
p_unit1["alternative"]="CfD"
p_unit1["is_active"]="no"
p_unit=p_unit[p_unit['unit'].apply(lambda x: 1 if any(i in x for i in ["Solar","Wind Offshore"]) else 0)==1]
p_unit=p_unit[p_unit['unit'].apply(lambda x: 1 if any(i in x for i in ["New"]) else 0)==1]
p_unit["alternative"]="CfD_fix"
p_unit['eff00']=""
p_unit['availability']=""
p_unit['investMIP']=""
p_unit['maxUnitCount']=p_unit["capacity"]
p_unit['is_active']=""
p_unit['is_active'][p_unit["maxUnitCount"]==0]="no"
p_unit['eff01']=""
p_unit['op00']=""
p_unit['op01']=""
p_unit=p_unit.rename(columns={"capacity": "minUnitCount"})
p_unit=pd.concat([p_unit,p_unit1])
p_unit=p_unit[['unit','alternative','eff00','availability','investMIP','maxUnitCount','is_active','eff01','op00','op01','minUnitCount']].reset_index(drop=True)

C:\ws\Temp\1\ipykernel_9664\2720419132.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_unit['is_active'][p_unit["maxUnitCount"]==0]="no"


In [21]:
output="C:\ws\TradeRES\TradeRES-Backbone-demo/backbone\input\TradeRES/excels/TradeRES_MD_input.xlsx"
with pd.ExcelWriter(output) as writer:  
  index.to_excel(writer, sheet_name='index', index=False)
  p_gnu_io_new.to_excel(writer, sheet_name='p_gnu_io', index=False)
  p_unit.to_excel(writer, sheet_name='p_unit', index=False) 
  p_groupPolicy.to_excel(writer, sheet_name='p_groupPolicy', index=False) 
  p_groupPolicyUnit.to_excel(writer, sheet_name='p_groupPolicyUnit', index=False) 
  group.to_excel(writer, sheet_name='group', index=False) 